In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-ibm-transpiler

*Tinatayang QPU usage: Wala (TANDAAN: Ang tutorial na ito ay hindi nagsasagawa ng mga jobs dahil nakatuon ito sa transpilation)*

## Background
Ang **Qiskit AI-powered transpiler service (QTS)** ay nagpapakilala ng mga optimization na batay sa machine learning sa routing at synthesis passes. Ang mga AI mode na ito ay idinisenyo upang harapin ang mga limitasyon ng tradisyonal na transpilation, lalo na para sa malalaking circuit at komplikadong hardware topology.

Simula **Hulyo 2025**, ang **Transpiler Service** ay inilipat na sa bagong IBM Quantum&reg; Platform at hindi na available. Para sa pinakabagong updates tungkol sa status ng Transpiler Service, mangyaring sumangguni sa [transpiler service documentation](/guides/qiskit-transpiler-service). Maaari pa rin ninyong gamitin ang AI transpiler nang lokal, katulad ng standard na Qiskit transpilation. Palitan lang ang `generate_preset_pass_manager()` ng `generate_ai_pass_manager()`. Ang function na ito ay bumubuo ng pass manager na nagsasama ng AI-powered routing at synthesis passes nang direkta sa inyong lokal na transpilation workflow.

### Mga pangunahing feature ng AI passes
- Routing passes: Ang AI-powered routing ay maaaring dynamic na mag-adjust ng mga qubit path batay sa partikular na circuit at backend, binabawasan ang pangangailangan para sa sobrang SWAP gates.
    - `AIRouting`: Layout selection at circuit routing

- Synthesis passes: Ang mga AI technique ay nag-optimize ng decomposition ng multi-qubit gates, binabawasan ang bilang ng two-qubit gates, na karaniwang mas madaling magkaroon ng error.
    - `AICliffordSynthesis`: Clifford gate synthesis
    - `AILinearFunctionSynthesis`: Linear function circuit synthesis
    - `AIPermutationSynthesis`: Permutation circuit synthesis
    - `AIPauliNetworkSynthesis`: Pauli Network circuit synthesis (available lamang sa Qiskit Transpiler Service, hindi sa lokal na environment)

- Paghahambing sa tradisyonal na transpilation: Ang standard na Qiskit transpiler ay isang matibay na tool na kayang hawakan ang malawak na hanay ng quantum circuits nang epektibo. Gayunpaman, kapag ang mga circuit ay lumalaki o ang mga hardware configuration ay nagiging mas kumplikado, ang mga AI passes ay maaaring magbigay ng karagdagang optimization gains. Sa pamamagitan ng paggamit ng learned models para sa routing at synthesis, ang QTS ay higit pang nagpapabuti ng mga circuit layout at binabawasan ang overhead para sa mahirap o malalaking quantum tasks.

Sinusuri ng tutorial na ito ang mga AI mode gamit ang routing at synthesis passes, inihahambing ang mga resulta sa tradisyonal na transpilation upang i-highlight kung saan nag-aalok ng performance gains ang AI.

Para sa higit pang detalye tungkol sa available na AI passes, tingnan ang [AI passes documentation](/guides/ai-transpiler-passes).

### Bakit gumamit ng AI para sa quantum circuit transpilation?
Habang lumalaki ang laki at komplikadong ng mga quantum circuit, ang tradisyonal na mga pamamaraan ng transpilation ay nahihirapang mag-optimize ng mga layout at bawasan ang gate counts nang epektibo. Ang mas malalaking circuit, lalo na ang mga kinabibilangan ng daan-daang qubit, ay nagdudulot ng makabuluhang hamon sa routing at synthesis dahil sa mga device constraint, limitadong connectivity, at qubit error rates.

Dito nag-aalok ng potensyal na solusyon ang AI-powered transpilation. Sa pamamagitan ng paggamit ng machine learning techniques, ang AI-powered transpiler sa Qiskit ay maaaring gumawa ng mas matalinong mga desisyon tungkol sa qubit routing at gate synthesis, na humahantong sa mas magandang optimization ng malalaking quantum circuits.

### Maikling benchmarking results
![Graph showing AI transpiler performance against Qiskit](../docs/images/tutorials/ai-transpiler-introduction/ai-transpiler-benchmarks.avif)

Sa mga benchmarking test, ang AI transpiler ay patuloy na gumawa ng mas mababaw at mas mataas na kalidad na mga circuit kumpara sa standard na Qiskit transpiler. Para sa mga test na ito, ginamit namin ang default pass manager strategy ng Qiskit, na naka-configure sa [`generate_preset_passmanager`]. Bagama't ang default strategy na ito ay madalas na epektibo, maaari itong mahirapan sa mas malalaki o mas kumplikadong circuit. Sa kabaligtaran, ang AI-powered passes ay nakamit ang average na 24% reduction sa two-qubit gate counts at 36% reduction sa circuit depth para sa malalaking circuit (100+ qubits) kapag nag-transpile sa heavy-hex topology ng IBM Quantum hardware. Para sa higit pang impormasyon tungkol sa mga benchmark na ito, sumangguni sa [blog](https://www.ibm.com/quantum/blog/qiskit-performance) na ito.

Sinusuri ng tutorial na ito ang mga pangunahing benepisyo ng AI passes at kung paano ito inihahambing sa tradisyonal na mga pamamaraan.

In [1]:
# This cell is hidden from users;
# it just disables a linting rule.
# ruff: noqa: F811

## Requirements

Bago simulan ang tutorial na ito, siguraduhing mayroon kayong sumusunod na naka-install:

* Qiskit SDK v1.0 o mas bago, na may [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) support
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 o mas bago
* Qiskit IBM&reg; Transpiler with AI local mode(`pip install 'qiskit-ibm-transpiler[ai-local-mode]'`)

## Setup

In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import efficient_su2, PermutationGate
from qiskit.synthesis.qft import synth_qft_full
from qiskit.circuit.random import random_circuit, random_clifford_circuit
from qiskit.transpiler import generate_preset_pass_manager, CouplingMap
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_transpiler import generate_ai_pass_manager
from qiskit.synthesis.permutation import (
    synth_permutation_depth_lnn_kms,
    synth_permutation_basic,
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import logging

seed = 42


# Used for generating permutation circuits in part two for comparison
def generate_permutation_circuit(width, pattern):
    circuit = QuantumCircuit(width)
    circuit.append(
        PermutationGate(pattern=pattern),
        qargs=range(width),
    )
    return circuit


# Creates a Bernstein-Vazirani circuit given the number of qubits
def create_bv_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits - 1)
    qc.x(num_qubits - 1)
    qc.h(qc.qubits)
    for i in range(num_qubits - 1):
        qc.cx(i, num_qubits - 1)
    qc.h(qc.qubits[:-1])
    return qc


# Transpile a circuit with a given pass manager and return metrics
def transpile_with_metrics(pass_manager, circuit):
    start = time.time()
    qc_out = pass_manager.run(circuit)
    elapsed = time.time() - start

    depth_2q = qc_out.depth(lambda x: x.operation.num_qubits == 2)
    gate_count = qc_out.size()

    return qc_out, {
        "depth_2q": depth_2q,
        "gate_count": gate_count,
        "time_s": elapsed,
    }


# Used for collecting metrics for part 3 of synthesis methods
def synth_transpile_with_metrics(qc, pm, pattern_id, method):
    start = time.time()
    qc = pm.run(qc)
    elapsed = time.time() - start

    return {
        "Pattern": pattern_id,
        "Method": method,
        "Depth (2Q)": qc.depth(lambda x: x.operation.num_qubits == 2),
        "Gates": qc.size(),
        "Time (s)": elapsed,
    }


# Ignore logs like "INFO:qiskit_ibm_transpiler.wrappers.ai_local_synthesis:Running Linear Functions AI synthesis on local mode"

logging.getLogger(
    "qiskit_ibm_transpiler.wrappers.ai_local_synthesis"
).setLevel(logging.WARNING)

# Part I. Qiskit patterns

Tingnan natin ngayon kung paano gamitin ang AI transpiler service sa isang simpleng quantum circuit, gamit ang Qiskit patterns. Ang susi ay ang paglikha ng `PassManager` gamit ang `generate_ai_pass_manager()` sa halip na ang standard na `generate_preset_pass_manager()`.

## Step 1: Map classical inputs to a quantum problem

Sa seksyong ito, susubukan natin ang AI transpiler sa `efficient_su2` circuit, isang malawakang ginagamit na hardware-efficient ansatz. Ang circuit na ito ay partikular na nauugnay para sa variational quantum algorithms (halimbawa, VQE) at quantum machine-learning tasks, ginagawa itong perpektong test case para sa pagsusuri ng transpilation performance.

Ang `efficient_su2` circuit ay binubuo ng alternating layers ng single-qubit rotations at entangling gates tulad ng CNOTs. Ang mga layer na ito ay nagbibigay-daan sa flexible na pagsisiyasat ng quantum state space habang pinapanatiling manageable ang gate depth. Sa pamamagitan ng pag-optimize sa circuit na ito, naglalayong nating bawasan ang gate count, pahusayin ang fidelity, at bawasan ang noise. Ginagawa itong malakas na kandidato para sa pagsubok sa efficiency ng AI transpiler.

In [3]:
# For our transpilation, we will use a large circuit of 101 qubits
qc = efficient_su2(90, entanglement="circular", reps=1).decompose()

# Draw a smaller version of the circuit to get a visual representation
qc_small = efficient_su2(5, entanglement="circular", reps=1).decompose()
qc_small.draw(output="mpl")

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif)

## Step 2: Optimize problem for quantum hardware execution
### Pumili ng backend
Para sa halimbawang ito, pipiliin natin ang pinakahindi busy operational IBM Quantum backend na hindi simulator at may kahit 100 qubits:

**Tandaan:** Dahil ang least-busy backend ay maaaring magbago sa paglipas ng panahon, iba't ibang mga device ang maaaring mapili para sa iba't ibang runs. Ang mga device-specific properties, tulad ng coupling maps, ay maaaring humantong sa mga pagkakaiba sa mga transpiled circuits.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=100
)
cm = backend.coupling_map
print(f"Using backend: {backend.name}")

Using backend: ibm_torino


### Create AI and traditional pass managers
To evaluate the effectiveness of the AI transpiler, we will perform two transpilation runs. First, we will transpile the circuit using the AI transpiler. Then, we will run a comparison by transpiling the same circuit without the AI transpiler, using traditional methods. Both transpilation processes will use the same coupling map from the chosen backend and the optimization level set to 3 for a fair comparison.

Both of these methods reflect the standard approach to create `PassManager` instances to transpile circuits in Qiskit.

In [5]:
pm_ai = generate_ai_pass_manager(
    optimization_level=3,
    ai_optimization_level=3,
    coupling_map=cm,
    include_ai_synthesis=True,  # used for part 3 when comparing synthesis methods
)

pm_no_ai = generate_preset_pass_manager(
    optimization_level=3,
    coupling_map=cm,
    seed_transpiler=seed,  # note that the AI pass manager does not currently support seeding
)

### Lumikha ng AI at tradisyonal na pass managers
Upang masuri ang pagiging epektibo ng AI transpiler, magsasagawa tayo ng dalawang transpilation runs. Una, i-transpile natin ang circuit gamit ang AI transpiler. Pagkatapos, magsasagawa tayo ng paghahambing sa pamamagitan ng pag-transpile sa parehong circuit nang walang AI transpiler, gamit ang tradisyonal na mga pamamaraan. Parehong mga proseso ng transpilation ay gagamit ng parehong coupling map mula sa napiling backend at ang optimization level na nakatakda sa 3 para sa patas na paghahambing.

Ang parehong mga pamamaraan na ito ay sumasalamin sa standard na approach para lumikha ng `PassManager` instances upang mag-transpile ng mga circuit sa Qiskit.

In [6]:
# Transpile using standard (non-AI) pass manager
_, metrics_no_ai = transpile_with_metrics(pm_no_ai, qc)
print(
    f"Standard transpilation: Depth (2q) {metrics_no_ai['depth_2q']}, "
    f"Gate count {metrics_no_ai['gate_count']}, Time {metrics_no_ai['time_s']}"
)

# Transpile using AI pass manager
_, metrics_ai = transpile_with_metrics(pm_ai, qc)
print(
    f"AI transpilation      : Depth (2q) {metrics_ai['depth_2q']}, "
    f"Gate count {metrics_ai['gate_count']}, Time {metrics_ai['time_s']}"
)

Standard transpilation: Depth (2q) 95, Gate count 458, Time 0.04650712013244629
AI transpilation      : Depth (2q) 90, Gate count 456, Time 0.9342479705810547


I-transpile ang mga circuit at itala ang mga oras.

In [7]:
# Circuits to benchmark
seed = 42
circuits = [
    {
        "name": "Random",
        "qc": random_circuit(num_qubits=30, depth=10, seed=seed),
    },
    {
        "name": "Clifford",
        "qc": random_clifford_circuit(
            num_qubits=40, num_gates=200, seed=seed
        ),
    },
    {
        "name": "QFT",
        "qc": synth_qft_full(num_qubits=20, do_swaps=False).decompose(),
    },
    {
        "name": "BV",
        "qc": create_bv_circuit(40),
    },
]

results = []

# Run the transpilation for each circuit and store the results
for circuit in circuits:
    qc_no_ai, metrics_no_ai = transpile_with_metrics(pm_no_ai, circuit["qc"])
    qc_ai, metrics_ai = transpile_with_metrics(pm_ai, circuit["qc"])

    print("Completed transpilation for", circuit["name"])

    results.append(
        {
            "Circuit": circuit["name"],
            "Depth 2Q (No AI)": metrics_no_ai["depth_2q"],
            "Gate Count (No AI)": metrics_no_ai["gate_count"],
            "Time (No AI)": metrics_no_ai["time_s"],
            "Depth 2Q (AI)": metrics_ai["depth_2q"],
            "Gate Count (AI)": metrics_ai["gate_count"],
            "Time (AI)": metrics_ai["time_s"],
        }
    )

df = pd.DataFrame(results)
df

Completed transpilation for Random
Completed transpilation for Clifford
Completed transpilation for QFT
Completed transpilation for BV


,Circuit,Depth 2Q (No AI),Gate Count (No AI),Time (No AI),Depth 2Q (AI),Gate Count (AI),Time (AI)
0,Random,37,221,0.039347,24,181,0.773718
1,Clifford,36,232,0.036633,43,267,1.097431
2,QFT,165,924,0.077458,130,913,3.660771
3,BV,65,155,0.024993,70,155,0.345522


Average percentage reduction for each metric. Positive are improvements, negative are degradations.

In [8]:
# Average reduction from non-AI to AI transpilation as a percentage
avg_reduction_depth = (
    (df["Depth 2Q (No AI)"] - df["Depth 2Q (AI)"]).mean()
    / df["Depth 2Q (No AI)"].mean()
    * 100
)
avg_reduction_gates = (
    (df["Gate Count (No AI)"] - df["Gate Count (AI)"]).mean()
    / df["Gate Count (No AI)"].mean()
    * 100
)
avg_reduction_time = (
    (df["Time (No AI)"] - df["Time (AI)"]).mean()
    / df["Time (No AI)"].mean()
    * 100
)

print(f"Average reduction in depth: {avg_reduction_depth:.2f}%")
print(f"Average reduction in gate count: {avg_reduction_gates:.2f}%")
print(f"Average reduction in transpilation time: {avg_reduction_time:.2f}%")

Average reduction in depth: 11.88%
Average reduction in gate count: 1.04%
Average reduction in transpilation time: -3193.95%


In [9]:
fig, axs = plt.subplots(1, 3, figsize=(21, 6))
df.plot(
    x="Circuit",
    y=["Depth 2Q (No AI)", "Depth 2Q (AI)"],
    kind="bar",
    ax=axs[0],
)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Circuit")
axs[0].tick_params(axis="x", rotation=45)
df.plot(
    x="Circuit",
    y=["Gate Count (No AI)", "Gate Count (AI)"],
    kind="bar",
    ax=axs[1],
)
axs[1].set_title("Gate Count Comparison")
axs[1].set_ylabel("Gate Count")
axs[1].set_xlabel("Circuit")
axs[1].tick_params(axis="x", rotation=45)
df.plot(x="Circuit", y=["Time (No AI)", "Time (AI)"], kind="bar", ax=axs[2])
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Circuit")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI transpilation vs Non-AI transpilation for various circuits"
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif" alt="Output of the previous code cell" />

The AI transpiler's performance varies significantly based on the type of circuit being optimized. In some cases, it achieves notable reductions in circuit depth and gate count compared to the standard transpiler. However, these improvements often come with a substantial increase in runtime.

For certain types of circuits, the AI transpiler may yield slightly better results in terms of circuit depth but may also lead to an increase in gate count and a significant runtime penalty. These observations suggest that the AI transpiler's benefits are not uniform across all circuit types. Instead, its effectiveness depends on the specific characteristics of the circuit, making it more suitable for some use cases than others.

## When should users choose AI-powered transpilation?

The AI-powered transpiler in Qiskit excels in scenarios where traditional transpilation methods struggle, particularly with large-scale and complex quantum circuits. For circuits involving hundreds of qubits or those targeting hardware with intricate coupling maps, the AI transpiler offers superior optimization in terms of circuit depth, gate count, and runtime efficiency. In benchmarking tests, it has consistently outperformed traditional methods, delivering significantly shallower circuits and reducing gate counts, which are critical for enhancing performance and mitigating noise on real quantum hardware.

Users should consider AI-powered transpilation when working with:
- Large circuits where traditional methods fail to efficiently handle the scale.
- Complex hardware topologies where device connectivity and routing challenges arise.
- Performance-sensitive applications where reducing circuit depth and improving fidelity are paramount.

# Part III. Explore AI-powered permutation network synthesis

Permutation networks are foundational in quantum computing, particularly for systems constrained by restricted topologies. These networks facilitate long-range interactions by dynamically swapping qubits to mimic all-to-all connectivity on hardware with limited connectivity. Such transformations are essential for implementing complex quantum algorithms on near-term devices, where interactions often span beyond nearest neighbors.

In this section, we highlight the synthesis of permutation networks as a compelling use case for the AI-powered transpiler in Qiskit. Specifically, the `AIPermutationSynthesis` pass leverages AI-driven optimization to generate efficient circuits for qubit permutation tasks. By contrast, generic synthesis approaches often struggle to balance gate count and circuit depth, especially in scenarios with dense qubit interactions or when attempting to achieve full connectivity.

We will walk through a Qiskit patterns example showcasing the synthesis of a permutation network to achieve all-to-all connectivity for a set of qubits. We will compare the performance of `AIPermutationSynthesis` against the standard synthesis methods in Qiskit. This example will demonstrate how the AI transpiler optimizes for lower circuit depth and gate count, highlighting its advantages in practical quantum workflows. To activate the AI synthesis pass, we will use the `generate_ai_pass_manager()` function with the `include_ai_synthesis` parameter set to `True`.

## Step 1: Map classical inputs to a quantum problem

To represent a classical permutation problem on a quantum computer, we start by defining the structure of the quantum circuits. For this example:

1. Quantum circuit initialization:
   We allocate 27 qubits to match the backend we will use, which has 27 qubits.

2. Apply permutations:
   We generate ten random permutation patterns (`pattern_1` through `pattern_10`) using a fixed seed for reproducibility. Each permutation pattern is applied to a separate quantum circuit (`qc_1` through `qc_10`).

3. Circuit decomposition:
   Each permutation operation is decomposed into native gate sets compatible with the target quantum hardware. We analyze the depth and the number of two-qubit gates (nonlocal gates) for each decomposed circuit.

The results provide insight into the complexity of representing classical permutation problems on a quantum device, demonstrating the resource requirements for different permutation patterns.

In [10]:
# Parameters
width = 27
num_circuits = 10

# Set random seed
np.random.seed(seed)


# Generate random patterns and circuits
patterns = [
    np.random.permutation(width).tolist() for _ in range(num_circuits)
]
circuits = {
    f"qc_{i}": generate_permutation_circuit(width, pattern)
    for i, pattern in enumerate(patterns, start=1)
}

# Display one of the circuits
circuits["qc_1"].decompose(reps=3).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/76a3e847-0808-4413-bd0c-c760cd2df3f4-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution
In this step, we proceed with optimization using the AI synthesis passes.

For the AI synthesis passes, the `PassManager` requires only the coupling map of the backend. However, it is important to note that not all coupling maps are compatible; only those that the `AIPermutationSynthesis` pass has been trained on will work. Currently, the `AIPermutationSynthesis` pass supports blocks of sizes 65, 33, and 27 qubits. For this example we use a 27-qubit QPU.

For comparison, we will evaluate the performance of AI synthesis against generic permutation synthesis methods in Qiskit, including:

- `synth_permutation_depth_lnn_kms`: This method synthesizes a permutation circuit for a linear nearest-neighbor (LNN) architecture using the Kutin, Moulton, and Smithline (KMS) algorithm. It guarantees a circuit with a depth of at most $ n $ and a size of at most $ n(n-1)/2 $, where both depth and size are measured in terms of SWAP gates.

- `synth_permutation_basic`: This is a straightforward implementation that synthesizes permutation circuits without imposing constraints on connectivity or optimization for specific architectures. It serves as a baseline for comparing performance with more advanced methods.

Each of these methods represents a distinct approach to synthesizing permutation networks, providing a comprehensive benchmark against the AI-powered methods.

For more details about synthesis methods in Qiskit, refer to the [Qiskit API documentation](/docs/api/qiskit/synthesis).

Define the coupling map representing the 27-qubit QPU.

In [11]:
coupling_map = [
    [1, 0],
    [2, 1],
    [3, 2],
    [3, 5],
    [4, 1],
    [6, 7],
    [7, 4],
    [7, 10],
    [8, 5],
    [8, 9],
    [8, 11],
    [11, 14],
    [12, 10],
    [12, 13],
    [12, 15],
    [13, 14],
    [16, 14],
    [17, 18],
    [18, 15],
    [18, 21],
    [19, 16],
    [19, 22],
    [20, 19],
    [21, 23],
    [23, 24],
    [25, 22],
    [25, 24],
    [26, 25],
]
CouplingMap(coupling_map).draw()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/84dff2c2-a496-4828-bb8e-08d373816a36-0.avif" alt="Output of the previous code cell" />

Average percentage reduction para sa bawat metric. Ang positibo ay mga pagpapabuti, ang negatibo ay mga pagkasira.

In [12]:
results = []
pm_no_ai_synth = generate_preset_pass_manager(
    coupling_map=cm,
    optimization_level=1,  # set to 1 since we are using the synthesis methods
)

# Transpile and analyze all circuits
for i, (qc_name, qc) in enumerate(circuits.items(), start=1):
    pattern = patterns[i - 1]  # Get the corresponding pattern

    qc_depth_lnn_kms = synth_permutation_depth_lnn_kms(pattern)
    qc_basic = synth_permutation_basic(pattern)

    # AI synthesis
    results.append(
        synth_transpile_with_metrics(
            qc.decompose(reps=3),
            pm_ai,
            qc_name,
            "AI",
        )
    )

    # Depth-LNN-KMS Method
    results.append(
        synth_transpile_with_metrics(
            qc_depth_lnn_kms.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Depth-LNN-KMS",
        )
    )

    # Basic Method
    results.append(
        synth_transpile_with_metrics(
            qc_basic.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Basic",
        )
    )


results_df = pd.DataFrame(results)

Record the metrics (depth, gate count, time) for each circuit after transpilation.

In [13]:
# Calculate averages for each metric
average_metrics = results_df.groupby("Method")[
    ["Depth (2Q)", "Gates", "Time (s)"]
].mean()
average_metrics = average_metrics.round(3)  # Round to two decimal places
print("\n=== Average Metrics ===")
print(average_metrics)

# Identify the best non-AI method based on least average depth
non_ai_methods = [
    method for method in results_df["Method"].unique() if method != "AI"
]
best_non_ai_method = average_metrics.loc[non_ai_methods][
    "Depth (2Q)"
].idxmin()
print(
    f"\nBest Non-AI Method (based on least average depth): {best_non_ai_method}"
)

# Compare AI to the best non-AI method
ai_metrics = average_metrics.loc["AI"]
best_non_ai_metrics = average_metrics.loc[best_non_ai_method]

comparison = {
    "Metric": ["Depth (2Q)", "Gates", "Time (s)"],
    "AI": [
        ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"],
        ai_metrics["Time (s)"],
    ],
    best_non_ai_method: [
        best_non_ai_metrics["Depth (2Q)"],
        best_non_ai_metrics["Gates"],
        best_non_ai_metrics["Time (s)"],
    ],
    "Improvement (AI vs Best Non-AI)": [
        ai_metrics["Depth (2Q)"] - best_non_ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"] - best_non_ai_metrics["Gates"],
        ai_metrics["Time (s)"] - best_non_ai_metrics["Time (s)"],
    ],
}

comparison_df = pd.DataFrame(comparison)
print("\n=== Comparison of AI vs Best Non-AI Method ===")
comparison_df


=== Average Metrics ===
               Depth (2Q)  Gates  Time (s)
Method                                    
AI                   23.9   82.8     0.248
Basic                29.8   91.0     0.012
Depth-LNN-KMS        70.8  531.6     0.017

Best Non-AI Method (based on least average depth): Basic

=== Comparison of AI vs Best Non-AI Method ===


,Metric,AI,Basic,Improvement (AI vs Best Non-AI)
0,Depth (2Q),23.900,29.800,-5.900
1,Gates,82.800,91.000,-8.200
2,Time (s),0.248,0.012,0.236


The results demonstrate that the AI transpiler outperforms all other Qiskit synthesis methods for this set of random permutation circuits. Key findings include:

1. Depth: The AI transpiler achieves the lowest average depth, indicating superior optimization of circuit layouts.
2. Gate count: It significantly reduces the number of gates compared to other methods, improving execution fidelity and efficiency.
3. Transpilation time: All methods run very quickly at this scale, making them practical for use. However, the AI transpiler does has a notable runtime increase compared to traditional methods due to the complexity of the AI models used.

These results establish the AI transpiler as the most effective approach for this benchmark, particularly for depth and gate count optimization.

Plot the results to compare the performance of the AI synthesis passes against the generic synthesis methods.

In [14]:
methods = results_df["Method"].unique()

fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Pivot the DataFrame and reorder columns to ensure AI is first
pivot_depth = results_df.pivot(
    index="Pattern", columns="Method", values="Depth (2Q)"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_gates = results_df.pivot(
    index="Pattern", columns="Method", values="Gates"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_time = results_df.pivot(
    index="Pattern", columns="Method", values="Time (s)"
)[["AI", "Depth-LNN-KMS", "Basic"]]

pivot_depth.plot(kind="bar", ax=axs[0], legend=False)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Pattern")
axs[0].tick_params(axis="x", rotation=45)
pivot_gates.plot(kind="bar", ax=axs[1], legend=False)
axs[1].set_title("2Q Gate Count Comparison")
axs[1].set_ylabel("Number of 2Q Gates")
axs[1].set_xlabel("Pattern")
axs[1].tick_params(axis="x", rotation=45)
pivot_time.plot(
    kind="bar", ax=axs[2], legend=True, title="Legend"
)  # Show legend on the last plot
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Pattern")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI Synthesis Methods vs Non-AI Synthesis Methods For Random Permutations Circuits",
    fontsize=16,
    y=1,
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/a326f268-0115-442c-8563-968676b66670-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif)

Ang performance ng AI transpiler ay nag-iiba nang malaki batay sa uri ng circuit na ino-optimize. Sa ilang mga kaso, nakakamit nito ang kapansin-pansing mga pagbabawas sa circuit depth at gate count kumpara sa standard na transpiler. Gayunpaman, ang mga pagpapabuting ito ay madalas na dumarating na may makabuluhang pagtaas sa runtime.

Para sa ilang uri ng mga circuit, ang AI transpiler ay maaaring magbunga ng bahagyang mas magandang mga resulta sa mga tuntunin ng circuit depth ngunit maaari rin itong humantong sa pagtaas ng gate count at makabuluhang runtime penalty. Ang mga obserbasyon na ito ay nagmumungkahi na ang mga benepisyo ng AI transpiler ay hindi pare-pareho sa lahat ng uri ng circuit. Sa halip, ang pagiging epektibo nito ay nakadepende sa mga partikular na katangian ng circuit, ginagawa itong mas angkop para sa ilang use cases kaysa sa iba.

## Kailan dapat pumili ang mga user ng AI-powered transpilation?

Ang AI-powered transpiler sa Qiskit ay nangunguna sa mga sitwasyon kung saan ang tradisyonal na mga pamamaraan ng transpilation ay nahihirapan, partikular sa malalaki at kumplikadong quantum circuits. Para sa mga circuit na kinabibilangan ng daan-daang qubit o yaong nakatuon sa hardware na may masalimuot na coupling maps, ang AI transpiler ay nag-aalok ng superior optimization sa mga tuntunin ng circuit depth, gate count, at runtime efficiency. Sa mga benchmarking test, patuloy nitong nalampasan ang tradisyonal na mga pamamaraan, naghahatid ng mas mababaw na mga circuit at binabawasan ang mga gate count, na kritikal para sa pagpapahusay ng performance at pagpapagaan ng noise sa tunay na quantum hardware.

Dapat isaalang-alang ng mga user ang AI-powered transpilation kapag gumagawa ng:
- Malalaking circuit kung saan ang tradisyonal na mga pamamaraan ay nabibigo na hawakan nang epektibo ang scale.
- Kumplikadong hardware topology kung saan lumilitaw ang mga hamon sa device connectivity at routing.
- Performance-sensitive applications kung saan ang pagbawas ng circuit depth at pagpapabuti ng fidelity ay napakahalaga.

# Part III. Explore AI-powered permutation network synthesis

Ang mga permutation network ay pundasyon sa quantum computing, partikular para sa mga sistema na pinipigilan ng restricted topology. Ang mga network na ito ay nagpapadali ng long-range interactions sa pamamagitan ng dynamic na pag-swap ng mga qubit upang gayahin ang all-to-all connectivity sa hardware na may limitadong connectivity. Ang gayong mga transformation ay mahalaga para sa pagpapatupad ng kumplikadong quantum algorithms sa near-term devices, kung saan ang mga interaction ay madalas na lumalampas sa nearest neighbors.

Sa seksyong ito, binibigyang-diin natin ang synthesis ng permutation networks bilang nakakahimok na use case para sa AI-powered transpiler sa Qiskit. Partikular, ang `AIPermutationSynthesis` pass ay gumagamit ng AI-driven optimization upang bumuo ng efficient na mga circuit para sa qubit permutation tasks. Sa kabaligtaran, ang generic synthesis approaches ay madalas na nahihirapang balansehin ang gate count at circuit depth, lalo na sa mga sitwasyon na may dense qubit interactions o kapag sinusubukang makamit ang buong connectivity.

Lalakarin natin ang isang Qiskit patterns example na nagpapakita ng synthesis ng permutation network upang makamit ang all-to-all connectivity para sa isang set ng mga qubit. Ihahambing natin ang performance ng `AIPermutationSynthesis` laban sa standard na mga synthesis methods sa Qiskit. Ipapakita ng halimbawang ito kung paano nag-optimize ang AI transpiler para sa mas mababang circuit depth at gate count, binibigyang-diin ang mga bentahe nito sa praktikal na quantum workflows. Upang i-activate ang AI synthesis pass, gagamitin natin ang `generate_ai_pass_manager()` function na may `include_ai_synthesis` parameter na nakatakda sa `True`.

## Step 1: Map classical inputs to a quantum problem

Upang katawanin ang classical permutation problem sa quantum computer, magsisimula tayo sa pamamagitan ng pagtukoy sa istruktura ng mga quantum circuit. Para sa halimbawang ito:

1. Quantum circuit initialization:
   Naglalaan tayo ng 27 qubits upang tumugma sa backend na gagamitin natin, na may 27 qubits.

2. Mag-apply ng mga permutation:
   Bumubuo tayo ng sampung random permutation patterns (`pattern_1` hanggang `pattern_10`) gamit ang fixed seed para sa reproducibility. Ang bawat permutation pattern ay inilalapat sa hiwalay na quantum circuit (`qc_1` hanggang `qc_10`).

3. Circuit decomposition:
   Ang bawat permutation operation ay dine-decompose sa native gate sets na compatible sa target na quantum hardware. Sinusuri natin ang depth at bilang ng two-qubit gates (nonlocal gates) para sa bawat decomposed circuit.

Ang mga resulta ay nagbibigay ng insight sa komplikadong ng pagkatawan ng classical permutation problems sa quantum device, ipinapakita ang mga pangangailangan sa resources para sa iba't ibang permutation patterns.